## ___Toxic Tweets Fine-Tuned Pretrained Transformer with Multi Head Classifier___

In [ ]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from transformers import Trainer, TrainingArguments

In [2]:
# If GPU is available, use it, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Getting training dataset
df = pd.read_csv("../data/raw/train.csv")
# Comments as list of strings for training texts
texts = df["comment_text"].tolist()
# Labels extracted from dataframe as list of lists
labels = df[["toxic","severe_toxic","obscene","threat","insult","identity_hate"]].values.tolist()
# Training set split into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2)

In [3]:
# Tokenizing training and validation sets
class ToxicTweetsDataset(Dataset):
    # Initialize the class variables
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    # Returns the length of the dataset
    def __len__(self):
        return len(self.encodings['input_ids'])
    # Returns a dictionary of the tokenized text, attention mask, and labels
    def __getitem__(self, index):
        input_ids = self.encodings['input_ids'][index]
        attention_mask = self.encodings['attention_mask'][index]
        labels = torch.tensor(self.labels[index], dtype=torch.float32)
        return {'input_ids': input_ids,
                'attention_mask': attention_mask,
                'labels': labels}

In [ ]:
# Choosing base pretrained model
model_name = "distilbert-base-uncased"
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)

In [ ]:
# Tokenizing and encoding training and validation sets
train_encodings = tokenizer.batch_encode_plus(train_texts, truncation=True, padding=True, return_tensors='pt')
val_encodings = tokenizer.batch_encode_plus(val_texts, truncation=True, padding=True, return_tensors='pt')

In [ ]:
# # Saving encoded and tokenized data to files
# torch.save(train_encodings, '../data/tokenized_encodings/train_encodings.pt')
# torch.save(val_encodings, '../data/tokenized_encodings/val_encodings.pt')

In [ ]:
# # Loading encoded and tokenized data from files
# train_encodings = torch.load('../data/tokenized_encodings/train_encodings.pt').to(device)
# val_encodings = torch.load('../data/tokenized_encodings/val_encodings.pt').to(device)

In [ ]:
# Creating datasets for training and validation
train_dataset = ToxicTweetsDataset(train_encodings, train_labels)
val_dataset = ToxicTweetsDataset(val_encodings, val_labels)

# Creating model
model = DistilBertForSequenceClassification.from_pretrained(model_name, problem_type="multi_label_classification", num_labels=6)
model.to(device)

### Training Setup & Process

In [ ]:
# Setting training arguments
training_args = TrainingArguments(
    output_dir="../models/fine_tuned",
    num_train_epochs=2, 
    per_device_train_batch_size=16, 
    per_device_eval_batch_size=32, 
    warmup_steps=500,
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch",
    save_total_limit=1,
)

# Creating trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Training model
trainer.train()

### Testing The Fine-Tuned Model


In [4]:
# Loading test dataset
test_size = 10000
test_df = pd.read_csv("../data/raw/test.csv")
test_label_df = pd.read_csv("../data/raw/test_labels.csv")

# Comments as list of strings for testing texts
test_texts = test_df["comment_text"].tolist()[:test_size]
# Labels extracted from dataframe as list of lists
test_labels = test_label_df[["toxic","severe_toxic","obscene","threat","insult","identity_hate"]].values.tolist()[:test_size]

In [5]:
# Loading model and tokenizer
model = DistilBertForSequenceClassification.from_pretrained("sergey-hovhannisyan/fine-tuned-toxic-tweets")
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")
model.eval()

# Tokenizing and encoding test set
test_encodings = tokenizer.batch_encode_plus(test_texts, truncation=True, padding=True, return_tensors='pt')

# Creating datasets for testing set
test_dataset = ToxicTweetsDataset(test_encodings, test_labels)

This code sets a batch size for evaluation, creates a DataLoader object for the test dataset, and then iterates over the batches in the test set. For each batch, it moves the data to the GPU (if available), uses the trained model to make predictions, and then appends the batch predictions and labels to two lists. Finally, it combines all batch predictions and labels into one array each.

In [6]:
# Batch size for evaluation
batch_size = 32

# Create the DataLoader for our test set
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Create lists for preds and labels
all_preds = []
all_labels = []

# Loop over batches
for batch in test_loader:
    # move batch to GPU if available
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        # make predictions
        outputs = model(**batch)
        logits = outputs.logits
        preds = torch.sigmoid(logits)
        preds = (preds > 0.5).int()
    # append predictions and labels to lists
    all_preds.append(preds.cpu().numpy())
    all_labels.append(batch['labels'].cpu().numpy())

# Combine all predictions and labels
all_preds = np.concatenate(all_preds, axis=0)
all_labels = np.concatenate(all_labels, axis=0)

In the current scenario, we are only evaluating the "toxic" label column since the test label dataset assigns a value of -1 to all labels if any form of toxicity is detected.

In [7]:
# Getting only toxic label from predictions & labels
all_preds_toxic = -1*all_preds[:,0]
all_labels_toxic = all_labels[:,0]

In [8]:
# Calculating metrics
def compute_metrics(labels, predictions):
    accuracy = accuracy_score(labels, predictions)
    precision = precision_score(labels, predictions, average='weighted')
    recall = recall_score(labels, predictions, average='weighted')
    f1 = f1_score(labels, predictions, average='weighted')
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1 }

In [ ]:
# calculate evaluation metrics
metrics = compute_metrics(all_labels_toxic, all_preds_toxic)

# print evaluation metrics
print('Recall: ', round(metrics['recall'],4))
print('Precision: ', round(metrics['precision'],4))
print('Accuracy: ', round(metrics['accuracy'],4))
print('F1: ', round(metrics['f1'],4))